In [1]:
# Dependencies
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
# Make results reproducible
#seed = 1235
#np.random.seed(seed)
#tf.set_random_seed(seed)

In [3]:
#Loading the dataset
X = np.load('male_array_1.npy')
data_Y = np.load('male_array_2.npy')
#X = X[:300000]
#data_Y = data_Y[:300000]
data_Y = pd.DataFrame(data_Y)
data_Y.columns = ['혈당', '수축기혈압', '이완기혈압', '총콜레스테롤','HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백']
#Y = pd.get_dummies(data_Y['혈당'])
#y = np.array(Y)


In [4]:
label = '이완기혈압'

X = pd.DataFrame(X)
y = data_Y[label]

In [5]:

X[label] = y
n = len(X[X[label] == 4])
temp = pd.concat([X[X[label] == 1][:n], X[X[label] == 2][:n], X[X[label] == 3][:n], X[X[label]==4]], axis = 0)
y = pd.get_dummies(temp[label])
y = np.array(y)
X = np.array(temp[[0,1,2,3,4]])
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]], dtype=uint8)

In [6]:
X

array([[12.        , 10.94408733, 10.00639853, 10.02204825,  2.        ],
       [ 9.        , 10.19802596,  8.70392301,  7.7133806 ,  1.        ],
       [10.        , 10.94408733, 10.00639853, 10.57173103,  3.        ],
       ...,
       [10.        , 10.94408733,  9.57224002,  9.36242892,  3.        ],
       [ 9.        ,  9.45196458, 10.00639853,  9.9121117 ,  3.        ],
       [15.        ,  9.45196458,  9.57224002, 11.78103313,  3.        ]])

In [7]:
# Creating a Train and a Test Dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [8]:
# Session
sess = tf.Session()

In [9]:
# Interval / Epochs
interval = 100
epoch = 1000

In [10]:
# Initialize placeholders
X_data = tf.placeholder(shape=[None, 5], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 4], dtype=tf.float32)

In [11]:
input_layer_nodes = 5
hidden_layer_nodes1 = 30
hidden_layer_nodes2 = 30
hidden_layer_nodes3 = 30
output_layer_nodes =  4


In [12]:
import keras

keep_prob = tf.placeholder("float")


w1 = tf.get_variable("w1", shape=[input_layer_nodes, hidden_layer_nodes1], initializer=tf.keras.initializers.he_normal())
b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes1]))   # First Bias


w2 = tf.get_variable("w2", shape=[hidden_layer_nodes1, hidden_layer_nodes2], initializer=tf.keras.initializers.he_normal())
b2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes2]))   # Second Bias


w3 = tf.get_variable("w3", shape=[hidden_layer_nodes2, hidden_layer_nodes3], initializer=tf.keras.initializers.he_normal())
b3 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes3]))


w4 = tf.get_variable("w4", shape=[hidden_layer_nodes3, output_layer_nodes], initializer=tf.keras.initializers.he_normal())
b4 = tf.Variable(tf.random_normal(shape=[output_layer_nodes]))










Using TensorFlow backend.


In [13]:
# Operations
hidden_output1 = tf.nn.relu(tf.add(tf.matmul(X_data, w1), b1))
hidden_output1 = tf.nn.dropout(hidden_output1, keep_prob = keep_prob)

hidden_output2 = tf.nn.relu(tf.add(tf.matmul(hidden_output1, w2), b2))
hidden_output2 = tf.nn.dropout(hidden_output2, keep_prob = keep_prob)

hidden_output3 = tf.nn.relu(tf.add(tf.matmul(hidden_output2, w3), b3))
hidden_output3 = tf.nn.dropout(hidden_output3, keep_prob = keep_prob)

logits = tf.add(tf.matmul(hidden_output3, w4), b4)
final_output = tf.nn.softmax(logits)


In [14]:
# Cost Function
#loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=0))
##
##
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_target))

In [15]:
# Optimizer
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [16]:
# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

In [17]:
# Training
print('Training the model...')
for i in range(1, (epoch + 1)):
    sess.run(optimizer, feed_dict={X_data: X_train, y_target: y_train, keep_prob : 0.5})
    if i % interval == 0:
        print('Epoch', i, '|', 'Loss:', sess.run(loss, feed_dict={X_data: X_train, y_target: y_train, keep_prob : 0.5}))


Training the model...
('Epoch', 100, '|', 'Loss:', 1.4120805)
('Epoch', 200, '|', 'Loss:', 1.3854162)
('Epoch', 300, '|', 'Loss:', 1.3779947)
('Epoch', 400, '|', 'Loss:', 1.3715439)
('Epoch', 500, '|', 'Loss:', 1.3681449)
('Epoch', 600, '|', 'Loss:', 1.3622519)
('Epoch', 700, '|', 'Loss:', 1.3576634)
('Epoch', 800, '|', 'Loss:', 1.353138)
('Epoch', 900, '|', 'Loss:', 1.3533245)
('Epoch', 1000, '|', 'Loss:', 1.3516688)


In [18]:
# Prediction
#print()
#for i in range(len(X_test)):
#    print('Actual:', y_test[i], 'Predicted:', (sess.run(final_output, feed_dict={X_data: [X_test[i]], keep_prob : 1.0})))

In [22]:
count = 0
count2 = 0
count3 = 0
for i in range(len(X_test)):
    if np.argmax(y_test[i]) in [1,2,3]:
        count3 += 1
    if np.argmax(y_test[i]) == np.argmax(sess.run(final_output, feed_dict={X_data: [X_test[i]], keep_prob:1.0})):
        count+=1
        if np.argmax(y_test[i]) in [1,2,3]:
            count2 += 1
print('accuracy : ', float(count)/len(X_test), 'accuracy2 : ',float(count2)/count3)

('accuracy : ', 0.3275560929732635, 'accuracy2 : ', 0.24191380700654064)


In [20]:
len(X_test)

14886

In [21]:
a

NameError: name 'a' is not defined